# Notebook: Train Model

## Packages

In [1]:
from load_dataset_folds import load_dataset_folds
from ACD import train_ACD_model
from OTE import train_OTE_model
import pandas as pd
import numpy as np
import constants
import warnings
import shutil
import torch
import json
import sys

* Todo: "output2" umbenennen ✅
* Todo: Evaluationsskript anpassen ✅
* Todo: "OTD" entfernen ✅
* Todo: Metrics speichern ✅
* Todo: Pfade ändern real/fake data ✅
* Todo: Nochmal prüfen, ob korrekte daten geladen werden
* Todo: Schauen, wieso der erste Term mehrere Tags erhält
* Todo: F1 macro brechnen
* Metriken bei acd checken und schauen, ob evtl standardmethoden verwendet werden können

## Parameters

In [2]:
LLM_NAME = "Llama13B"
N_REAL = 500
N_SYNTH = 0
TARGET = "aspect_category" # "aspect_term", "aspect_category"
LLM_SAMPLING = "fixed"

## Settings

In [3]:
# Set seeds
torch.manual_seed(constants.RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(constants.RANDOM_SEED)

# Ignore warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.optimization")
torch.device("mps")

# Disable Pycache
sys.dont_write_bytecode = True

## Code

In [4]:
train_dataset, test_dataset = load_dataset_folds(LLM_NAME, N_REAL, N_SYNTH, LLM_SAMPLING)

### Load Model

In [5]:
if TARGET == "aspect_category":
   results = train_ACD_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/32 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.5692, 'learning_rate': 0.0, 'epoch': 1.0}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.5192740559577942, 'eval_hamming_loss': 0.226, 'eval_accuracy': 0.064, 'eval_f1_macro': 0.10222222222222224, 'eval_f1_micro': 0.19630156472261737, 'eval_f1_weighted': 0.1620399579390116, 'eval_class_f1_scores': [0.0, 0.0, 0.5111111111111112, 0.0, 0.0], 'eval_runtime': 2.1098, 'eval_samples_per_second': 236.992, 'eval_steps_per_second': 15.167, 'epoch': 1.0}
{'train_runtime': 14.7514, 'train_samples_per_second': 33.895, 'train_steps_per_second': 2.169, 'train_loss': 0.5691688656806946, 'epoch': 1.0}


  0%|          | 0/32 [00:00<?, ?it/s]

In [6]:
if TARGET == "aspect_term":
    results = train_OTE_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

### Save Results

In [7]:
with open(f'results_json/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.json', 'w') as json_file:
    json.dump(results, json_file)

df = pd.DataFrame([results])
df.to_csv(f'results_csv/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.csv', index=False)

In [8]:
results

{'LLM_NAME': 'Llama13B',
 'N_REAL': 500,
 'N_SYNTH': 0,
 'TARGET': 'aspect_category',
 'LLM_SAMPLING': 'fixed',
 'loss': 0.5192740559577942,
 'hamming': 0.226,
 'accuracy': 0.064,
 'f1_micro': 0.19630156472261737,
 'f1_macro': 0.10222222222222224,
 'f1_weighted': 0.1620399579390116,
 'runtime': 21.08582091331482,
 'runtime_formatted': '21s'}

### Remove useless folders

In [9]:
try:
    shutil.rmtree("outputs")
except:
    pass